In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

from pyspark.sql.functions import col, when, to_timestamp, unix_timestamp
from pyspark.sql.types import DoubleType

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
from pyspark.sql.types import (StructField,StructType, BooleanType, DoubleType,LongType, IntegerType)
from pyspark.sql.types import *

In [3]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [4]:
# Définition du type : SPOINT

spoint_schema = [StructField('lat', FloatType(),True),
                 StructField('lon', FloatType(),True)]
spoint = StructType(fields=spoint_schema)

In [5]:
# Définition du type : SECTION

section_schema = [StructField('lat1', FloatType(),True),
                  StructField('lon1', FloatType(),True),
                  StructField('lat2', FloatType(),True),
                  StructField('lon2', FloatType(),True)]
section = StructType(fields=section_schema)

In [6]:
# Définition du type : SLINE

sline_schema = [StructField('rints', ArrayType(section),True)]
sline = StructType(fields=sline_schema)

In [7]:
# Définition du type : USPOINT

uspoint_schema = [StructField('t1', LongType(),True),
                  StructField('t2', LongType(),True),
                  StructField('lat1', FloatType(),True),
                  StructField('lon1', FloatType(),True),
                  StructField('lat2', FloatType(),True),
                  StructField('lon2', FloatType(),True)]
uspoint = StructType(fields=uspoint_schema)

In [8]:
# Définition du type : MSPOINT

mspoint_schema = [StructField('rints', ArrayType(uspoint),True)]
mspoint = StructType(fields=mspoint_schema)

In [9]:
# Définition du type : UINT

uint_schema = [StructField('val', IntegerType(),True),
               StructField('t1', LongType(),True),
               StructField('t2', LongType(),True)]
uint = StructType(fields=uint_schema)

In [10]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [11]:
# Définition du type : MINT

mint_schema = [StructField('units', ArrayType(uint),True)]
mint = StructType(fields=mint_schema)

In [12]:
# Définition du type : USTRING

ustring_schema = [StructField('val', StringType(),True),
               StructField('t1', LongType(),True),
               StructField('t2', LongType(),True)]
ustring = StructType(fields=ustring_schema)

In [13]:
# Définition du type : MSTRING

mstring_schema = [StructField('units', ArrayType(ustring),True)]
mstring = StructType(fields=mstring_schema)

In [14]:
# Définition du type : UREAL

ureal_schema = [StructField('a', FloatType(),True),
              StructField('b', FloatType(),True),
              StructField('c', FloatType(),True),
              StructField('r', BooleanType(),True),
              StructField('t1', LongType(),True),
              StructField('t2', LongType(),True)]
ureal = StructType(fields=ureal_schema)

In [15]:
# Définition du type : MREAL

mreal_schema = [StructField('units', ArrayType(ureal),True)]
mreal = StructType(fields=mreal_schema)

In [16]:
# Définition du type : USINT

usint_schema = [StructField('val', IntegerType(),True),
              StructField('interval', section,True)]
usint = StructType(fields=usint_schema)

In [17]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [18]:
# Définition du type : MSINT

msint_schema = [StructField('units', ArrayType(usint),False)]
msint = StructType(fields=msint_schema)

In [19]:
# Définition du type : USSTRING

usstring_schema = [StructField('val', StringType(),True),
              StructField('interval', section,True)]
usstring = StructType(fields=usstring_schema)

In [20]:
# Définition du type : MSSTRING

msstring_schema = [StructField('units', ArrayType(usstring),False)]
msstring = StructType(fields=msstring_schema)

In [21]:
# Définition du type : USREAL

usreal_schema = [StructField('a', FloatType(),True),
                 StructField('b', FloatType(),True),
                 StructField('c', FloatType(),True),
                 StructField('r', BooleanType(),True),
                 StructField('interval', section,True)]
usreal = StructType(fields=usreal_schema)

In [22]:
# Définition du type : MSREAL

msreal_schema = [StructField('units', ArrayType(usreal),False)]
msreal = StructType(fields=msreal_schema)

In [23]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [24]:
# Définition du type : INTIME

intime_schema = [StructField('val', FloatType(),True),
                 StructField('t1', LongType(),True)]
intime = StructType(fields=intime_schema)

In [25]:
# Définition du type : INSPOINT

inspoint_schema = [StructField('val', FloatType(),True),
                   StructField('sp', spoint,True)]
inspoint = StructType(fields=inspoint_schema)

In [26]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [27]:
df = spark.read.option("header",True).option("inferSchema",True).csv("VGP-week3-data.csv")
df.printSchema()

df.persist(pyspark.storagelevel.StorageLevel(True, False, False, False))

root
 |-- kit_id: integer (nullable = true)
 |-- participant_id: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- PM2-5: string (nullable = true)
 |-- PM10: string (nullable = true)
 |-- PM1-0: string (nullable = true)
 |-- NO2: string (nullable = true)
 |-- BC: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- event: string (nullable = true)



DataFrame[kit_id: int, participant_id: int, time: string, lat: double, lon: double, PM2-5: string, PM10: string, PM1-0: string, NO2: string, BC: string, activity: string, event: string]

In [28]:
#On change les string "NULL" en null
df = df.withColumn(("BC"), when(col("BC") == "NULL", None).otherwise(col("BC")))
df = df.withColumn(("PM2-5"), when(col("PM2-5") == "NULL", None).otherwise(col("PM2-5")))
df = df.withColumn(("PM10"),  when(col("PM10") == "NULL",  None).otherwise(col("PM10")))
df = df.withColumn(("PM1-0"), when(col("PM1-0") == "NULL", None).otherwise(col("PM1-0")))
df = df.withColumn(("NO2"),   when(col("NO2") == "NULL",   None).otherwise(col("NO2")))
df = df.withColumn(("activity"), when(col("activity") == "NULL", None).otherwise(col("activity")))
df = df.withColumn(("event"), when(col("event") == "NULL", None).otherwise(col("event")))

In [29]:
df = df.withColumn('time',unix_timestamp('time', 'yyyy-MM-dd HH:mm:ss').alias('time'))
df.show()
df.printSchema()

+------+--------------+----------+----------------+----------------+-----+----+-----+----+----+--------+-----+
|kit_id|participant_id|      time|             lat|             lon|PM2-5|PM10|PM1-0| NO2|  BC|activity|event|
+------+--------------+----------+----------------+----------------+-----+----+-----+----+----+--------+-----+
|    80|       9999964|1573718400|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718410|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718420|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718430|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718440|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718450|48.7717766666667|        2.006005| null|null| null|null|null|    null| null|
|

In [30]:
#Conversion des types PM2.5, BC ... en double
#Remarque dans notre fichier de donnée j'ai eliminer le "+00" a chaque fois
df = df.withColumn("PM2-5",df["PM2-5"].cast(FloatType()))
df = df.withColumn("PM10",df["PM10"].cast(FloatType()))
df = df.withColumn("PM1-0",df["PM1-0"].cast(FloatType()))
df = df.withColumn("NO2",df["NO2"].cast(FloatType()))
df = df.withColumn("BC",df["BC"].cast(FloatType()))
df = df.withColumn("lat",df["lat"].cast(FloatType()))
df = df.withColumn("lon",df["lon"].cast(FloatType()))
df = df.withColumn("time",df["time"].cast(LongType()))
df.printSchema()

root
 |-- kit_id: integer (nullable = true)
 |-- participant_id: integer (nullable = true)
 |-- time: long (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)
 |-- PM2-5: float (nullable = true)
 |-- PM10: float (nullable = true)
 |-- PM1-0: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- BC: float (nullable = true)
 |-- activity: string (nullable = true)
 |-- event: string (nullable = true)



In [31]:
def change_column_names(columns):
    return [c.replace('-', '_') for c in columns]

df = df.toDF(*change_column_names(df.columns))
df.show()
df.count()

+------+--------------+----------+---------+---------+-----+----+-----+----+----+--------+-----+
|kit_id|participant_id|      time|      lat|      lon|PM2_5|PM10|PM1_0| NO2|  BC|activity|event|
+------+--------------+----------+---------+---------+-----+----+-----+----+----+--------+-----+
|    80|       9999964|1573718400| 48.77178| 2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718410| 48.77178| 2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718420| 48.77178| 2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718430| 48.77178| 2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718440| 48.77178| 2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718450| 48.77178| 2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718460| 48.77178| 2.006005| null|null| null|null|null|    null| null|
|    80|       9999964|1573718

59972

In [32]:
liste_id = df.select('participant_id').distinct().rdd.map(lambda r: r[0]).collect()

In [32]:
liste_id = [9999964]

In [33]:
data_type = StructType([StructField('id_participant', IntegerType(),True), StructField('trip', mspoint,True), StructField('PM2_5_t', mreal,True), StructField('PM_10_t', mreal,True), StructField('PM1_0_t', mreal,True), StructField('NO2_t', mreal,True), StructField('BC_t', mreal,True), StructField('activity_t', mstring,True), StructField('event_t', mstring,True)])
dataset = spark.createDataFrame(sc.emptyRDD(), data_type)
dataset.show()

+--------------+----+-------+-------+-------+-----+----+----------+-------+
|id_participant|trip|PM2_5_t|PM_10_t|PM1_0_t|NO2_t|BC_t|activity_t|event_t|
+--------------+----+-------+-------+-------+-----+----+----------+-------+
+--------------+----+-------+-------+-------+-----+----+----------+-------+



In [36]:
df = df.where(df['PM2_5'].isNotNull() & df['PM1_0'].isNotNull() & df['PM10'].isNotNull() & df['NO2'].isNotNull() & df['BC'].isNotNull() & df['activity'].isNotNull() & df['event'].isNotNull())
# new_participant_row = spark.createDataFrame(sc.emptyRDD(), StructType([StructField('id_participant', IntegerType(),True), StructField('trip', mspoint,True), StructField('PM2_5_t', mreal,True),StructField('PM_10_t', mreal,True), StructField('PM1_0_t', mreal,True), StructField('NO2_t', mreal,True), StructField('BC_t', mreal,True), StructField('activity_t', mstring,True), StructField('event_t', mstring,True),StructField('PM2_5_t', mreal,True), StructField('PM_10_s', msreal,True), StructField('PM1_0_s', msreal,True), StructField('NO2_s', msreal,True), StructField('BC_s', msreal,True), StructField('activity_s', msstring,True), StructField('event_s', msstring,True)]))
new_participant_row = spark.createDataFrame(sc.emptyRDD(), StructType([StructField('id_participant', IntegerType(),True), StructField('trip', mspoint,True), StructField('PM2_5_t', mreal,True),StructField('PM_10_t', mreal,True), StructField('PM1_0_t', mreal,True), StructField('NO2_t', mreal,True), StructField('BC_t', mreal,True), StructField('activity_t', mstring,True), StructField('event_t', mstring,True)]))    
new_participant_row.persist(pyspark.storagelevel.StorageLevel(True, False, False, False))
for id_participant in liste_id:   
        
    #MOUCHARD 1
    print("participant avec id: {}".format(id_participant))
    
    # Prendre toutes les données d'un seul participant et les trier par le temps
    df_one_participant = df.where(col("participant_id") == id_participant)
    df_one_participant = df_one_participant.sort("time")
    df_one_participant.persist(pyspark.storagelevel.StorageLevel(True, False, False, False))
    
#     #initialiser les df temporaires
#     trip_temp = spark.createDataFrame(sc.emptyRDD(), mspoint)
#     PM2_5_temp = spark.createDataFrame(sc.emptyRDD(), mreal)
#     PM10_temp = spark.createDataFrame(sc.emptyRDD(), mreal)
#     PM1_0_temp = spark.createDataFrame(sc.emptyRDD(), mreal)
#     NO2_temp = spark.createDataFrame(sc.emptyRDD(), mreal)
#     BC_temp = spark.createDataFrame(sc.emptyRDD(), mreal)
#     activity_temp = spark.createDataFrame(sc.emptyRDD(), mstring)
#     event_temp = spark.createDataFrame(sc.emptyRDD(), mstring)
#     PM2_5_s_temp = spark.createDataFrame(sc.emptyRDD(), msreal)
#     PM10_s_temp = spark.createDataFrame(sc.emptyRDD(), msreal)
#     PM1_0_s_temp = spark.createDataFrame(sc.emptyRDD(), msreal)
#     NO2_s_temp = spark.createDataFrame(sc.emptyRDD(), msreal)
#     BC_s_temp = spark.createDataFrame(sc.emptyRDD(), msreal)
#     activity_s_temp = spark.createDataFrame(sc.emptyRDD(), msstring)
#     event_s_temp = spark.createDataFrame(sc.emptyRDD(), msstring)

    trip_temp = sc.emptyRDD()
    PM2_5_temp =sc.emptyRDD()
    PM10_temp = sc.emptyRDD()
    PM1_0_temp = sc.emptyRDD()
    NO2_temp = sc.emptyRDD()
    BC_temp = sc.emptyRDD()
    activity_temp = sc.emptyRDD()
    event_temp = sc.emptyRDD()
    
    
    # Prendre les premieres données
    row = df_one_participant.first()
    print(df_one_participant.printSchema())
    kit_id = row.kit_id
    time=row.time+3600*id_participant
    lat=row.lat
    lon=row.lon
    PM2_5=row.PM2_5
    PM10=row.PM10
    PM1_0=row.PM1_0
    NO2=row.NO2
    BC=row.BC
    activity=row.activity
    event=row.event
    
    # Boucler toutes les données du participant
    index = 0
    size_df_participant = df_one_participant.count()
    
    while (index < size_df_participant):
        row = df_one_participant.collect()[index]
        print(row)

        if (row.time+3600*id_participant == time):
            time = row.time+3600*id_participant-1
        t_temp = row.time+3600*id_participant;
        
        
        # Mettre les données dans les df temporaires
        
        PM2_5_temp = PM2_5_temp.union(sc.parallelize([[([(0.0, (row.PM2_5-PM2_5)/(t_temp-time), (PM2_5*t_temp-row.PM2_5*time)/(t_temp-time), False, time, t_temp)])]]))
        PM2_5_temp.persist()
        PM10_temp = PM10_temp.union(sc.parallelize([[([(0.0, (row.PM10-PM10)/(t_temp-time), (PM10*t_temp-row.PM10*time)/(t_temp-time), False, time, t_temp)])]]))
        PM10_temp.persist()
        PM1_0_temp = PM1_0_temp.union(sc.parallelize([[([(0.0, (row.PM1_0-PM1_0)/(t_temp-time), (PM1_0*t_temp-row.PM1_0*time)/(t_temp-time), False, time, t_temp)])]]))
        PM1_0_temp.persist()
        NO2_temp = NO2_temp.union(sc.parallelize([[([(0.0, (row.NO2-NO2)/(t_temp-time), (NO2*t_temp-row.NO2*time)/(t_temp-time), False, time, t_temp)])]]))
        NO2_temp.persist()
        BC_temp = BC_temp.union(sc.parallelize([[([(0.0, (row.BC-BC)/(t_temp-time), (BC*t_temp-row.BC*time)/(t_temp-time), False, time, t_temp)])]]))
        BC_temp.persist()
        activity_temp = activity_temp.union(sc.parallelize([[([(activity, time, t_temp)])]]))
        activity_temp.persist()
        event_temp = event_temp.union(sc.parallelize([[([(row.event, time, t_temp)])]]))
        event_temp.persist()
        
        trip_temp = trip_temp.union(sc.parallelize([[([(time, t_temp, lat, lon, row.lat, row.lon)])]]))
        trip_temp.persist()
#         PM2_5_s_temp = PM2_5_s_temp.union([df_s_pm2_5])
#         PM10_s_temp = PM10_s_temp.union([df_s_pm10])
#         PM1_0_s_temp = PM1_0_s_temp.union([df_s_pm1_0])
#         NO2_s_temp = NO2_s_temp.union([df_s_no2])
#         BC_s_temp = BC_s_temp.union([df_s_bc])
#         activity_s_temp = activity_s_temp.union([df_s_activity])
#         event_s_temp = event_s_temp.union([de_s_event])
#         print("mreal pm2_5:")
#         PM2_5_temp.show()
        
        # Décallage des variables
        kit_id = row.kit_id
        time=row.time+3600*id_participant
        lat=row.lat
        lon=row.lon
        PM2_5=row.PM2_5
        PM10=row.PM10
        PM1_0=row.PM1_0
        NO2=row.NO2
        BC=row.BC
        activity=row.activity
        event=row.event
        
        # Passer à la ligne suivante
        #df_one_participant = df_one_participant.where(col("time") > row.time)
        index = index + 1
        spark.catalog.clearCache()
        
    # FIN WHILE
    
    df_one_participant.unpersist()    
#     d = sc.parallelize([id_participant, trip_temp, PM2_5_temp, PM10_temp, PM1_0_temp, NO2_temp, BC_temp, activity_temp, event_temp, PM2_5_s_temp, PM10_s_temp, PM1_0_s_temp, NO2_s_temp, BC_s_temp, activity_s_temp, event_s_temp ])
#     new_participant_row = spark.createDataFrame(d, StructType([StructField('id_participant', IntegerType(),True), StructField('trip', mspoint,True), StructField('PM2_5_t', mreal,True), StructField('PM_10_t', mreal,True), StructField('PM1_0_t', mreal,True), StructField('NO2_t', mreal,True), StructField('BC_t', mreal,True), StructField('activity_t', mstring,True), StructField('event_t', mstring,True),StructField('PM2_5_t', mreal,True), StructField('PM_10_s', msreal,True), StructField('PM1_0_s', msreal,True), StructField('NO2_s', msreal,True), StructField('BC_s', msreal,True), StructField('activity_s', msstring,True), StructField('event_s', msstring,True)]))
    print('okkkkkkkk')
    d = [(id_participant, trip_temp, PM2_5_temp, PM10_temp, PM1_0_temp, NO2_temp, BC_temp, activity_temp, event_temp)]
    new_participant_row = spark.createDataFrame(d, StructType([StructField('id_participant', IntegerType(),True), StructField('trip', mspoint,True), StructField('PM2_5_t', mreal,True), StructField('PM_10_t', mreal,True), StructField('PM1_0_t', mreal,True), StructField('NO2_t', mreal,True), StructField('BC_t', mreal,True), StructField('activity_t', mstring,True), StructField('event_t', mstring,True)]))
    
    dataset = dataset.union(new_participant_row)
# FIN DU FOR
dataset.show()
dataset.toPandas().to_csv('dataset.csv')

participant avec id: 9999964
root
 |-- kit_id: integer (nullable = true)
 |-- participant_id: integer (nullable = true)
 |-- time: long (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)
 |-- PM2_5: float (nullable = true)
 |-- PM10: float (nullable = true)
 |-- PM1_0: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- BC: float (nullable = true)
 |-- activity: string (nullable = true)
 |-- event: string (nullable = true)

None
Row(kit_id=80, participant_id=9999964, time=1573804980, lat=48.771728515625, lon=2.0058634281158447, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=7.0, BC=376.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573804990, lat=48.771728515625, lon=2.0058634281158447, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=7.0, BC=376.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573805000, lat=48.771728515625, lon=2.0058634281158447, PM2_5=3.0, PM10=4.0,

Row(kit_id=80, participant_id=9999964, time=1573809630, lat=48.77177047729492, lon=2.0058584213256836, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=7.0, BC=476.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573809640, lat=48.77177047729492, lon=2.0058584213256836, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=7.0, BC=476.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573809650, lat=48.77177047729492, lon=2.0058584213256836, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=7.0, BC=476.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573810380, lat=48.77175521850586, lon=2.006096601486206, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=8.0, BC=456.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573810390, lat=48.77175521850586, lon=2.006096601486206, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=8.0, BC=456.0, activity='Bureau', event='Arrêter De Cui

Row(kit_id=80, participant_id=9999964, time=1573815080, lat=48.771728515625, lon=2.005976676940918, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=5.0, BC=457.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573815090, lat=48.771728515625, lon=2.005976676940918, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=5.0, BC=457.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573815100, lat=48.771728515625, lon=2.005976676940918, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=5.0, BC=457.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573815110, lat=48.771785736083984, lon=2.0058467388153076, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=5.0, BC=457.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573815660, lat=48.77183151245117, lon=2.0058300495147705, PM2_5=2.0, PM10=2.0, PM1_0=2.0, NO2=5.0, BC=442.0, activity='Bureau', event='Arrêter De Cuisiner'

Row(kit_id=80, participant_id=9999964, time=1573816030, lat=48.77177429199219, lon=2.0059683322906494, PM2_5=2.0, PM10=2.0, PM1_0=2.0, NO2=6.0, BC=442.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573816040, lat=48.77177429199219, lon=2.0059683322906494, PM2_5=2.0, PM10=2.0, PM1_0=2.0, NO2=6.0, BC=442.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573816050, lat=48.77177429199219, lon=2.0059683322906494, PM2_5=2.0, PM10=2.0, PM1_0=2.0, NO2=6.0, BC=442.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573816060, lat=48.77177429199219, lon=2.0059683322906494, PM2_5=2.0, PM10=2.0, PM1_0=2.0, NO2=6.0, BC=442.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573816070, lat=48.77177429199219, lon=2.0059683322906494, PM2_5=2.0, PM10=2.0, PM1_0=2.0, NO2=6.0, BC=442.0, activity='Bureau', event='Arrêter De C

Row(kit_id=80, participant_id=9999964, time=1573816500, lat=48.771766662597656, lon=2.005913257598877, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=5.0, BC=459.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573816510, lat=48.771766662597656, lon=2.005913257598877, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=5.0, BC=459.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573816520, lat=48.771766662597656, lon=2.005913257598877, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=5.0, BC=459.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573816530, lat=48.771766662597656, lon=2.005913257598877, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=5.0, BC=459.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573816540, lat=48.771766662597656, lon=2.005913257598877, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=5.0, BC=459.0, activity='Bureau', event='Arrêter De C

...

Row(kit_id=80, participant_id=9999964, time=1573818730, lat=48.771793365478516, lon=2.0058815479278564, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=7.0, BC=433.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573818740, lat=48.771793365478516, lon=2.0058815479278564, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=7.0, BC=433.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573818750, lat=48.771793365478516, lon=2.0058815479278564, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=7.0, BC=433.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573818760, lat=48.771793365478516, lon=2.0058815479278564, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=7.0, BC=433.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573818770, lat=48.771793365478516, lon=2.0058815479278564, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=7.0, BC=433.0, activity='Bureau', event='Arrêter

Row(kit_id=80, participant_id=9999964, time=1573819140, lat=48.7717399597168, lon=2.0059916973114014, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=6.0, BC=432.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573819150, lat=48.7717399597168, lon=2.0059916973114014, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=6.0, BC=432.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573819160, lat=48.7717399597168, lon=2.0059916973114014, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=6.0, BC=432.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573819170, lat=48.7717399597168, lon=2.0059916973114014, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=6.0, BC=432.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573819180, lat=48.771705627441406, lon=2.0058634281158447, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=6.0, BC=432.0, activity='Bureau', event='Arrêter De Cuis

Row(kit_id=80, participant_id=9999964, time=1573819610, lat=48.771785736083984, lon=2.006023406982422, PM2_5=2.0, PM10=3.0, PM1_0=3.0, NO2=6.0, BC=450.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573819620, lat=48.771785736083984, lon=2.006023406982422, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=7.0, BC=422.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573819630, lat=48.771785736083984, lon=2.006023406982422, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=7.0, BC=422.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573819640, lat=48.771785736083984, lon=2.006023406982422, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=7.0, BC=422.0, activity='Bureau', event='Arrêter De Cuisiner')
Row(kit_id=80, participant_id=9999964, time=1573819650, lat=48.771785736083984, lon=2.006023406982422, PM2_5=3.0, PM10=3.0, PM1_0=3.0, NO2=7.0, BC=422.0, activity='Bureau', event='Arrêter De C

Row(kit_id=80, participant_id=9999964, time=1573820020, lat=48.771793365478516, lon=2.005894899368286, PM2_5=3.0, PM10=4.0, PM1_0=4.0, NO2=7.0, BC=441.0, activity='Bureau', event='Arrêter De Cuisiner')


KeyboardInterrupt: 

In [37]:
spark.catalog.clearCache()

In [93]:
data_type = StructType([StructField('id_participant', IntegerType(),True), StructField('trip', mspoint,True), StructField('PM2_5_t', mreal,True), StructField('PM_10_t', mreal,True), StructField('PM1_0_t', mreal,True), StructField('NO2_t', mreal,True), StructField('BC_t', mreal,True), StructField('activity_t', mstring,True), StructField('event_t', mstring,True)])
dataset = spark.createDataFrame(sc.emptyRDD(), data_type)
dataset.show()

trip_temp = spark.createDataFrame(sc.emptyRDD(), mspoint)
PM2_5_temp = spark.createDataFrame(sc.emptyRDD(), mreal)
PM10_temp = spark.createDataFrame(sc.emptyRDD(), mreal)
PM1_0_temp = spark.createDataFrame(sc.emptyRDD(), mreal)
NO2_temp = spark.createDataFrame(sc.emptyRDD(), mreal)
BC_temp = spark.createDataFrame(sc.emptyRDD(), mreal)
activity_temp = spark.createDataFrame(sc.emptyRDD(), mstring)
event_temp = spark.createDataFrame(sc.emptyRDD(), mstring)

PM2_5_temp = PM2_5_temp.union(spark.createDataFrame([[([(0.0, 0.0, 2.0, False, 0, 1)])]], mreal))
PM10_temp = PM10_temp.union(spark.createDataFrame([[([(0.0, 0.0, 2.0, False, 0, 1)])]], mreal))
PM1_0_temp = PM1_0_temp.union(spark.createDataFrame([[([(0.0, 0.0, 2.0, False, 0, 1)])]], mreal))
NO2_temp = NO2_temp.union(spark.createDataFrame([[([(0.0, 0.0, 2.0, False, 0, 1)])]], mreal))
BC_temp = BC_temp.union(spark.createDataFrame([[([(0.0, 0.0, 2.0, False, 0, 1)])]], mreal))
activity_temp = activity_temp.union(spark.createDataFrame([[([("home", 0, 1)])]], mstring))
event_temp = event_temp.union(spark.createDataFrame([[([("cooking", 0, 1)])]], mstring))
trip_temp = trip_temp.union(spark.createDataFrame([[([(0, 1, 4.5, 2.0, 4.8, 2.6)])]], mspoint))
trip_temp = trip_temp.union(spark.createDataFrame([[([(1, 2, 8.5, 4.0, 8.8, 5.6)])]], mspoint))

d = [(id_participant, trip_temp.rdd.collect()[0], PM2_5_temp.collect()[0], PM10_temp.collect()[0], PM1_0_temp.collect()[0], NO2_temp.collect()[0], BC_temp.collect()[0], activity_temp.collect()[0], event_temp.collect()[0])]
new_participant_row = spark.createDataFrame(d, StructType([StructField('id_participant', IntegerType(),True), StructField('trip', mspoint,True), StructField('PM2_5_t', mreal,True), StructField('PM_10_t', mreal,True), StructField('PM1_0_t', mreal,True), StructField('NO2_t', mreal,True), StructField('BC_t', mreal,True), StructField('activity_t', mstring,True), StructField('event_t', mstring,True)]))

print("dataframe new_participant_row")
new_participant_row.show()

dataset = dataset.union(new_participant_row)
print("dataframe dataset")
dataset.show()


+--------------+----+-------+-------+-------+-----+----+----------+-------+
|id_participant|trip|PM2_5_t|PM_10_t|PM1_0_t|NO2_t|BC_t|activity_t|event_t|
+--------------+----+-------+-------+-------+-----+----+----------+-------+
+--------------+----+-------+-------+-------+-----+----+----------+-------+

dataframe new_participant_row
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------------+
|id_participant|                trip|             PM2_5_t|             PM_10_t|             PM1_0_t|               NO2_t|                BC_t|      activity_t|            event_t|
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------------+
|       9999964|[[[0, 1, 4.5, 2.0...|[[[0.0, 0.0, 2.0,...|[[[0.0, 0.0, 2.0,...|[[[0.0, 0.0, 2.0,...|[[[0.0, 0.0, 2.0,...|[[[0

In [117]:
# New Cell Amir
df = df.where(df['PM2_5'].isNotNull() & df['PM1_0'].isNotNull() & df['PM10'].isNotNull() & df['NO2'].isNotNull() & df['BC'].isNotNull() & df['activity'].isNotNull() & df['event'].isNotNull())
# new_participant_row = spark.createDataFrame(sc.emptyRDD(), StructType([StructField('id_participant', IntegerType(),True), StructField('trip', mspoint,True), StructField('PM2_5_t', mreal,True),StructField('PM_10_t', mreal,True), StructField('PM1_0_t', mreal,True), StructField('NO2_t', mreal,True), StructField('BC_t', mreal,True), StructField('activity_t', mstring,True), StructField('event_t', mstring,True),StructField('PM2_5_t', mreal,True), StructField('PM_10_s', msreal,True), StructField('PM1_0_s', msreal,True), StructField('NO2_s', msreal,True), StructField('BC_s', msreal,True), StructField('activity_s', msstring,True), StructField('event_s', msstring,True)]))
new_participant_row = spark.createDataFrame(sc.emptyRDD(), StructType([StructField('id_participant', IntegerType(),True), StructField('trip', mspoint,True), StructField('PM2_5_t', mreal,True),StructField('PM_10_t', mreal,True), StructField('PM1_0_t', mreal,True), StructField('NO2_t', mreal,True), StructField('BC_t', mreal,True), StructField('activity_t', mstring,True), StructField('event_t', mstring,True)]))    
for id_participant in liste_id : 
    #MOUCHARD 1
    print("participant avec id: {}".format(id_participant))
    
    # Prendre toutes les données d'un seul participant et les trier par le temps
    df_one_participant = df.where(col("participant_id") == id_participant)
    df_one_participant = df_one_participant.sort("time")
        
    # A partir de la je reprend ma methode
    
    #Creation d'une nouvelle DF temp pour faire la jointure et regrouper les tuples (1 avec 2 , 3 avec 4, ect...)
    temp = df_one_participant
    #On renome les colonnes de temp pour qu'il y'ait pas d'ambiguité
    temp = temp.withColumnRenamed('kit_id','kit_id_temp')
    temp = temp.withColumnRenamed('participant_id','participant_id_temp')
    temp = temp.withColumnRenamed('time','time_temp')
    temp = temp.withColumnRenamed('lat','lat_temp')
    temp = temp.withColumnRenamed('lon','lon_temp')
    temp = temp.withColumnRenamed('PM2_5','PM2_5_temp')
    temp = temp.withColumnRenamed('PM10','PM10_temp')
    temp = temp.withColumnRenamed('PM1_0','PM1_0_temp')
    temp = temp.withColumnRenamed('NO2','NO2_temp')
    temp = temp.withColumnRenamed('BC','BC_temp')
    temp = temp.withColumnRenamed('activity','activity_temp')
    temp = temp.withColumnRenamed('event','event_temp')
    
    #temp.show()
    #On decremente le time de temp pour pouvoir faire la jointure avec == (car j'ai pas pu le faire avec == time_temp + 10)
    temp = temp.withColumn('time_temp', temp.time_temp - 10)
    #On fait la jointure
    #On fais le rename ici car y'avais un bug de spark 
    df_one_participant = df_one_participant.withColumnRenamed('time','time')
    x = df_one_participant.join(temp, [temp['time_temp'] == df_one_participant['time'] ])
    # On decremente le time_temp pour revenir a un etat cohérent
    y = x.select(x.lat,x.lon,x['time'],x['time_temp'] + 10, x.lat_temp,x.lon_temp, x['PM2_5'], x['PM2_5_temp'], x['PM10'], x['PM10_temp'], x['PM1_0'], x['PM1_0_temp'], x['NO2'], x['NO2_temp'], x['BC'], x['BC_temp'], x['activity'], x['activity_temp'], x['event'], x['event_temp'])
        
    # Mettre les données dans les df temporaires    

    rdd_pm25 = y.rdd.map(lambda r :[[(0.0, ((r[7]-r[6])/(r[3]-r[2])), (((r[6]*r[3])-(r[7]*r[2]))/((r[3]-r[2]))),False,r[2],r[3])]])
    rdd_pm10 = y.select(y.columns).rdd.map(lambda r : [[(0.0, ((r[9]-r[8])/(r[3]-r[2])), (((r[8]*r[3])-(r[9]*r[2]))/((r[3]-r[2]))),False,r[2],r[3])]])
    rdd_pm1_0 = y.select(y.columns).rdd.map(lambda r :[[(0.0, ((r[11]-r[10])/(r[3]-r[2])), (((r[10]*r[3])-(r[11]*r[2]))/((r[3]-r[2]))), False, r[2], r[3])]])
    rdd_no2 = y.select(y.columns).rdd.map(lambda r :  [[(0.0, ((r[13]-r[12])/(r[3]-r[2])), (((r[12]*r[3])-(r[13]*r[2]))/((r[3]-r[2]))), False, r[2], r[3])]])
    rdd_bc = y.select(y.columns).rdd.map(lambda r :   [[(0.0, ((r[15]-r[14])/(r[3]-r[2])), (((r[14]*r[3])-(r[15]*r[2]))/((r[3]-r[2]))), False, r[2], r[3])]])
    rdd_activity = y.select(y.columns).rdd.map(lambda r : [[(r[16], r[2], r[3])]])
    rdd_event = y.select(y.columns).rdd.map(lambda r : [[(r[18], r[2], r[3])]])
    rdd_trip = y.select(y.columns).rdd.map(lambda r : [[(r[2], r[3], r[0], r[1], r[4], r[5])]])
    
    trip_df_temp = spark.createDataFrame(rdd_trip, mspoint)
    PM2_5_df_temp = spark.createDataFrame(rdd_pm25,mreal)
    PM10_df_temp = spark.createDataFrame(rdd_pm10, mreal)
    PM1_0_df_temp = spark.createDataFrame(rdd_pm1_0, mreal)
    NO2_df_temp = spark.createDataFrame(rdd_no2, mreal)
    BC_df_temp = spark.createDataFrame(rdd_bc, mreal)
    activity_df_temp = spark.createDataFrame(rdd_activity, mstring)
    event_df_temp = spark.createDataFrame(rdd_event, mstring)
    

participant avec id: 9999964


In [105]:
print (rdd_pm25.take(5))

[[(0.0, 0.0, 3.0, False, 1573804980, 1573804990)], [(0.0, 0.0, 3.0, False, 1573804990, 1573805000)], [(0.0, 0.0, 3.0, False, 1573805000, 1573805010)], [(0.0, 0.0, 3.0, False, 1573805010, 1573805020)], [(0.0, 0.0, 3.0, False, 1573805020, 1573805030)]]


In [131]:
activity_df_temp.show()
PM2_5_df_temp.printSchema()

#trip_df_temp.show()

+--------------------+
|               units|
+--------------------+
|[[Bureau, 1573804...|
|[[Bureau, 1573804...|
|[[Bureau, 1573805...|
|[[Bureau, 1573805...|
|[[Bureau, 1573805...|
|[[Bureau, 1573805...|
|[[Bureau, 1573805...|
|[[Bureau, 1573805...|
|[[Bureau, 1573805...|
|[[Bureau, 1573805...|
|[[Bureau, 1573806...|
|[[Bureau, 1573806...|
|[[Bureau, 1573806...|
|[[Bureau, 1573806...|
|[[Bureau, 1573806...|
|[[Bureau, 1573807...|
|[[Bureau, 1573807...|
|[[Bureau, 1573807...|
|[[Bureau, 1573807...|
|[[Bureau, 1573807...|
+--------------------+
only showing top 20 rows

root
 |-- units: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- a: float (nullable = true)
 |    |    |-- b: float (nullable = true)
 |    |    |-- c: float (nullable = true)
 |    |    |-- r: boolean (nullable = true)
 |    |    |-- t1: long (nullable = true)
 |    |    |-- t2: long (nullable = true)



In [129]:
d = [(id_participant, trip_df_temp.rdd.collect()[0], PM2_5_df_temp.rdd.collect()[0], PM10_df_temp.rdd.collect()[0], PM1_0_df_temp.rdd.collect()[0], NO2_df_temp.rdd.collect()[0], BC_df_temp.rdd.collect()[0], activity_df_temp.rdd.collect()[0], event_df_temp.rdd.collect()[0])]
new_participant_row = spark.createDataFrame(d, StructType([StructField('id_participant', IntegerType(),True), StructField('trip', mspoint,True), StructField('PM2_5_t', mreal,True), StructField('PM_10_t', mreal,True), StructField('PM1_0_t', mreal,True), StructField('NO2_t', mreal,True), StructField('BC_t', mreal,True), StructField('activity_t', mstring,True), StructField('event_t', mstring,True)]))

In [130]:
new_participant_row.show()

+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|id_participant|                trip|             PM2_5_t|             PM_10_t|             PM1_0_t|               NO2_t|                BC_t|          activity_t|             event_t|
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       9999964|[[[1573804980, 15...|[[[0.0, 0.0, 3.0,...|[[[0.0, 0.0, 4.0,...|[[[0.0, 0.0, 4.0,...|[[[0.0, 0.0, 7.0,...|[[[0.0, 0.0, 376....|[[[Bureau, 157380...|[[[Arrêter De Cui...|
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [125]:
PM10_df_temp.rdd.collect()

[Row(units=[Row(a=0.0, b=0.0, c=4.0, r=False, t1=1573804980, t2=1573804990)]),
 Row(units=[Row(a=0.0, b=0.0, c=4.0, r=False, t1=1573804990, t2=1573805000)]),
 Row(units=[Row(a=0.0, b=0.0, c=4.0, r=False, t1=1573805000, t2=1573805010)]),
 Row(units=[Row(a=0.0, b=0.0, c=4.0, r=False, t1=1573805010, t2=1573805020)]),
 Row(units=[Row(a=0.0, b=0.0, c=4.0, r=False, t1=1573805020, t2=1573805030)]),
 Row(units=[Row(a=0.0, b=0.0, c=3.0, r=False, t1=1573805760, t2=1573805770)]),
 Row(units=[Row(a=0.0, b=0.0, c=3.0, r=False, t1=1573805770, t2=1573805780)]),
 Row(units=[Row(a=0.0, b=0.0, c=3.0, r=False, t1=1573805780, t2=1573805790)]),
 Row(units=[Row(a=0.0, b=0.0, c=3.0, r=False, t1=1573805790, t2=1573805800)]),
 Row(units=[Row(a=0.0, b=0.0, c=3.0, r=False, t1=1573805800, t2=1573805810)]),
 Row(units=[Row(a=0.0, b=0.0, c=3.0, r=False, t1=1573806540, t2=1573806550)]),
 Row(units=[Row(a=0.0, b=0.0, c=3.0, r=False, t1=1573806550, t2=1573806560)]),
 Row(units=[Row(a=0.0, b=0.0, c=3.0, r=False, t1=157